# The Price Is Right

In order to examine the impacts of different cost functions on specific problems we are going to use an example from the gameshow The Price is Right. In the first round of The Price is Right, 4 contestants "Come on down!" to play against each other. They are given an item and asked to guess how much it costs. If the contestant goes over the price, they automatically lose. The winner is the contestant with the guess that is closest to the actual price without going over.  
  
So what if we want to maximize our chances of winning on The Price is Right? Well as a data scientist we might think that this is the perfect chance to train a model to make predictions for us. We need to build a model that gets as close to the correct price as possible without going over. We are going to explore how we can use cost functions to build the best model possible.  

We are using data from the Amazon Toy Dataset to train and test on. With the price as our target variable, I have taken the liberty of building our exogenous variables using NLP and some basic data cleaning on a set of descriptions, reviews, and information for each product.  

In this experiment we are going to use a very popular algorithm, [LightGBM from Microsoft](https://lightgbm.readthedocs.io/en/latest/). With LightGBM, we are going to explore several different functions to try to optimize our results. We are going to use the basic MAE and MSE along with some custom objective and evaluation functions of our own to see which one works best.  

To evaluate our "winnings" on The Price is Right, we are going to calculate them as follows. If a prediction is over the true value, our winnings are 0. If our prediction is below the true value, then our winnings are equal to our prediction. For example, if an item was worth $10 and we predicted it was worth 7, we would win 7 dollars. This will encourage us to not only predict under the actual value, but still predict as close to the value as possible.

In [ ]:
import pandas as pd
import numpy as np
from lightgbm import LGBMRegressor
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.metrics import make_scorer
from sklearn.ensemble import RandomForestRegressor

## Calculating Winnings
First we are going to write a class to calculate the winnings our model would make from its set of predictions. It will also tell us how many of the products our model overpredicted. If our model overpredicts, our winnings are zero. If our model underpredicts, then our winnings equal our prediction.  

We will also look at winnings as a percentage of each price we predict under. That will see how we fare when we don't weight more expensive items more heavily. This will give us multiple points of view for how our models are doing. It also could possibly represent someone who would rather win a big item vs. someone just interested in moving onto the second round.

In [138]:
class price_is_right:
    def __init__(self, truth, preds):
        self.truths = truth
        self.preds = preds
        
    def calculate_winnings(self):
        winning_df = pd.DataFrame(zip(self.truths, self.preds), columns=['truth', 'pred'])
        winning_df['winnings'] = 0
        winning_df['winning_percent'] = 0
        winning_df.loc[winning_df['pred'] <= winning_df['truth'], 'winnings'] = winning_df['pred']
        winning_df.loc[winning_df['pred'] <= winning_df['truth'], 'winning_percent'] = winning_df['pred']/winning_df['truth']
        self.winnings = winning_df['winnings'].sum()
        self.winning_percent = winning_df['winning_percent'].sum()
        self.overpredicted = winning_df[winning_df['winnings']==0].shape[0]
        self.over_percent = self.overpredicted/len(self.preds)
    
    def print_results(self):
        self.calculate_winnings()
        print(f'OVERPREDICTED: {self.overpredicted}/{len(self.preds)}')
        print(f'STANDARDIZED WINNINGS: {self.winning_percent}')
        print(f'TOTAL WINNINGS: {self.winnings}')

In [139]:
train = pd.read_csv('../data/amazon_train.csv')
test = pd.read_csv('../data/amazon_test.csv')

## Data
Let's take a quick overview of the data we are using. The price column is going to be our target. The manufacturer, seller, category, and subcategory columns have been encoded using [scikit-learns LabelEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html). The name, info, description, and reviews columns were created with word embeddings from [spacy](https://spacy.io/usage). If you are interested in how the data was prepared feel free to check out my [other notebook](../notebooks/data_cleaning.ipynb).

In [140]:
train.head()

,manufacturer,price,number_of_reviews,average_review_rating,sellers,cat,sub_cat,name,info,desc,reviews
0,987,8.44,2.0,5.0,549,34,14,0.605618,-1.305930,-1.993142,1.824300
1,1574,209.99,2.0,4.0,699,13,116,3.528920,-1.269589,-1.730483,-4.421397
2,1663,4.00,3.0,5.0,81,14,46,-3.511664,-0.837028,-2.297708,-3.458916
3,2342,19.99,1.0,5.0,160,7,140,0.836040,-1.123865,-2.583533,-2.375369
4,2105,1.49,77.0,4.5,387,0,97,1.504639,5.455936,-1.236510,-2.221026


We are going to split our test data into a validation and test set so that we have a validation set to help tune our model with. 

In [141]:
test, val = train_test_split(test, test_size=.25, random_state=0)
x_train = train.drop('price', 1)
y_train = train['price']
x_val = val.drop('price', 1)
y_val = val['price']
x_test = test.drop('price', 1)
y_test = test['price']

# LightGBM
Here is a quick helper function to help us run experiments and get the evaluations back. We will run them with all the same hyperparameters and with a random state set to try to make it as fair as possible. 

In [142]:
def run_experiment(x_train, y_train, x_val, y_val, x_test, y_test, obj, eval_met):
    lgb = LGBMRegressor(objective = obj,
                    subsample = 0.1,
                     random_state = 0,
                     num_leaves = 120,
                     n_estimators = 10000,
                     min_split_gain = 0.06999,
                     min_data_in_leaf = 12,
                     max_depth = 56,
                     learning_rate = 0.005,
                     colsample_bytree = 0.4,
                     boosting_type = 'gbdt')

    lgb.fit(x_train, y_train, eval_set=(x_val, y_val), eval_metric=eval_met, early_stopping_rounds=25, verbose=False)

    preds = lgb.predict(x_test)
    win = price_is_right(y_test, preds)
    return win

## Loss Functions
Here is the meat of what we are investigating. We are looking at how different loss functions will impact how our model makes predictions. The 2 functions that we will look at that are already supported by LightGBM are the [Mean Absolute Error](https://en.wikipedia.org/wiki/Mean_absolute_error) and the [Mean Squared Error](https://en.wikipedia.org/wiki/Mean_squared_error). We are also going to write a few of our own functions to try to get the most out of our model.  

With LightGBM there are 2 different types of functions that we need to write. The first one is an objective function for training. This function needs to be compatible with the [gradient descent](https://en.wikipedia.org/wiki/Gradient_descent) used to optimize the model. LightGBM requires objective functions to return both the first and second derivative of the cost function.  

The second type of function that we can use with LightGBM is an evaluation function. This function is how we will optimize our models parameters with our validation data during training. So while updating weights based on the objective function, the model will run until the score of our evaluation function stops improving. 

Most of these functions are going to be some variation of the MAE or MSE cost functions mentioned above. The difference is that we are going to add extra penalties if the model predicts above the true value. You will see that we are going to run these experiments several times with different penalties to see how the model is impacted.  

The last custom function we are going to look at is going to be very specific to our problem at hand. We are writing a function where the loss approximates our price is right evaluation. So it is trying to maximize our winnings from that specific game.   

So now let's see how the various models perform.

In [143]:
penalty = 3
def overprediction_penalty_objective(truth, pred):
    err = (truth - pred).astype("float")
    grad = np.where(err<0, -2*penalty*err, -2*err)
    hess = np.where(err<0, 2*penalty, 2)
    return grad, hess

def overprediction_penalty_squared(truth, pred):
    err = (truth - pred).astype("float")
    loss = np.where(err<0, (err**2)*penalty, err**2) 
    return "overprediction_squared_penalty", np.mean(loss), False

def overprediction_penalty_absolute(truth, pred):
    err = (truth - pred).astype("float")
    loss = np.where(err<0, abs(err)*penalty, abs(err)) 
    return "overprediction_squared_penalty", np.mean(loss), False

def overprediction_penalty_pr(truth, pred):
    err = (truth - pred).astype("float")
    loss = np.where(err<0, -pred, 0)
    return "custom_penalty", np.mean(loss), False

In [144]:
res_names = ['MAE', 'MSE', 'Asymmetric MAE Custom', 'Asymmetric MSE Custom', 'Asymmetric Price is Right Eval']
total_results = pd.DataFrame()

In [146]:
l1 = run_experiment(x_train, y_train, x_val, y_val, x_test, y_test, 'regression_l1', 'regression_l1')
l2 = run_experiment(x_train, y_train, x_val, y_val, x_test, y_test, 'regression_l2', 'regression_l1')
l1_custom = run_experiment(x_train, y_train, x_val, y_val, x_test, y_test,
                           overprediction_penalty_objective, overprediction_penalty_absolute)
l2_custom = run_experiment(x_train, y_train, x_val, y_val, x_test, y_test,
                           overprediction_penalty_objective, overprediction_penalty_squared)
pr_custom = run_experiment(x_train, y_train, x_val, y_val, x_test, y_test,
                           overprediction_penalty_objective, overprediction_penalty_custom)

temp = pd.DataFrame()
all_winnings = []
all_std_winnings = []
overpreds = []
results = [l1, l2, l1_custom, l2_custom, pr_custom]
for res, name in zip(results, res_names) :
    res.calculate_winnings()
    all_winnings.append(res.winnings)
    all_std_winnings.append(res.winning_percent)
    overpreds.append(res.over_percent)
temp['Cost'] = res_names
temp['Winnings'] = all_winnings
temp['Standardized_Winnings'] = all_std_winnings
temp['% OverPredicted'] = overpreds
temp['Penalty'] = penalty
total_results = pd.concat([total_results, temp], 0)

In [148]:
penalty = 5
def overprediction_penalty_objective(truth, pred):
    err = (truth - pred).astype("float")
    grad = np.where(err<0, -2*penalty*err, -2*err)
    hess = np.where(err<0, 2*penalty, 2)
    return grad, hess

def overprediction_penalty_squared(truth, pred):
    err = (truth - pred).astype("float")
    loss = np.where(err<0, (err**2)*penalty, err**2) 
    return "overprediction_squared_penalty", np.mean(loss), False

def overprediction_penalty_absolute(truth, pred):
    err = (truth - pred).astype("float")
    loss = np.where(err<0, abs(err)*penalty, abs(err)) 
    return "overprediction_squared_penalty", np.mean(loss), False

def overprediction_penalty_custom(truth, pred):
    err = (truth - pred).astype("float")
    loss = np.where(err<0, -pred, 0)
    return "custom_penalty", np.mean(loss), False

In [149]:
l1 = run_experiment(x_train, y_train, x_val, y_val, x_test, y_test, 'regression_l1', 'regression_l1')
l2 = run_experiment(x_train, y_train, x_val, y_val, x_test, y_test, 'regression_l2', 'regression_l1')
l1_custom = run_experiment(x_train, y_train, x_val, y_val, x_test, y_test,
                           overprediction_penalty_objective, overprediction_penalty_absolute)
l2_custom = run_experiment(x_train, y_train, x_val, y_val, x_test, y_test,
                           overprediction_penalty_objective, overprediction_penalty_squared)
pr_custom = run_experiment(x_train, y_train, x_val, y_val, x_test, y_test,
                           overprediction_penalty_objective, overprediction_penalty_custom)

temp = pd.DataFrame()
all_winnings = []
all_std_winnings = []
overpreds = []
results = [l1, l2, l1_custom, l2_custom, pr_custom]
for res, name in zip(results, res_names) :
    res.calculate_winnings()
    all_winnings.append(res.winnings)
    all_std_winnings.append(res.winning_percent)
    overpreds.append(res.over_percent)
temp['Cost'] = res_names
temp['Winnings'] = all_winnings
temp['Standardized_Winnings'] = all_std_winnings
temp['% OverPredicted'] = overpreds
temp['Penalty'] = penalty
total_results = pd.concat([total_results, temp], 0)

In [150]:
penalty = 10
def overprediction_penalty_objective(truth, pred):
    err = (truth - pred).astype("float")
    grad = np.where(err<0, -2*penalty*err, -2*err)
    hess = np.where(err<0, 2*penalty, 2)
    return grad, hess

def overprediction_penalty_squared(truth, pred):
    err = (truth - pred).astype("float")
    loss = np.where(err<0, (err**2)*penalty, err**2) 
    return "overprediction_squared_penalty", np.mean(loss), False

def overprediction_penalty_absolute(truth, pred):
    err = (truth - pred).astype("float")
    loss = np.where(err<0, abs(err)*penalty, abs(err)) 
    return "overprediction_squared_penalty", np.mean(loss), False

def overprediction_penalty_custom(truth, pred):
    err = (truth - pred).astype("float")
    loss = np.where(err<0, -pred, 0)
    return "custom_penalty", np.mean(loss), False

In [151]:
l1 = run_experiment(x_train, y_train, x_val, y_val, x_test, y_test, 'regression_l1', 'regression_l1')
l2 = run_experiment(x_train, y_train, x_val, y_val, x_test, y_test, 'regression_l2', 'regression_l1')
l1_custom = run_experiment(x_train, y_train, x_val, y_val, x_test, y_test,
                           overprediction_penalty_objective, overprediction_penalty_absolute)
l2_custom = run_experiment(x_train, y_train, x_val, y_val, x_test, y_test,
                           overprediction_penalty_objective, overprediction_penalty_squared)
pr_custom = run_experiment(x_train, y_train, x_val, y_val, x_test, y_test,
                           overprediction_penalty_objective, overprediction_penalty_custom)

temp = pd.DataFrame()
all_winnings = []
all_std_winnings = []
overpreds = []
results = [l1, l2, l1_custom, l2_custom, pr_custom]
for res, name in zip(results, res_names) :
    res.calculate_winnings()
    all_winnings.append(res.winnings)
    all_std_winnings.append(res.winning_percent)
    overpreds.append(res.over_percent)
temp['Cost'] = res_names
temp['Winnings'] = all_winnings
temp['Standardized_Winnings'] = all_std_winnings
temp['% OverPredicted'] = overpreds
temp['Penalty'] = penalty
total_results = pd.concat([total_results, temp], 0)

In [152]:
penalty = 12
def overprediction_penalty_objective(truth, pred):
    err = (truth - pred).astype("float")
    grad = np.where(err<0, -2*penalty*err, -2*err)
    hess = np.where(err<0, 2*penalty, 2)
    return grad, hess

def overprediction_penalty_squared(truth, pred):
    err = (truth - pred).astype("float")
    loss = np.where(err<0, (err**2)*penalty, err**2) 
    return "overprediction_squared_penalty", np.mean(loss), False

def overprediction_penalty_absolute(truth, pred):
    err = (truth - pred).astype("float")
    loss = np.where(err<0, abs(err)*penalty, abs(err)) 
    return "overprediction_squared_penalty", np.mean(loss), False

def overprediction_penalty_custom(truth, pred):
    err = (truth - pred).astype("float")
    loss = np.where(err<0, -pred, 0)
    return "custom_penalty", np.mean(loss), False

In [153]:
l1 = run_experiment(x_train, y_train, x_val, y_val, x_test, y_test, 'regression_l1', 'regression_l1')
l2 = run_experiment(x_train, y_train, x_val, y_val, x_test, y_test, 'regression_l2', 'regression_l1')
l1_custom = run_experiment(x_train, y_train, x_val, y_val, x_test, y_test,
                           overprediction_penalty_objective, overprediction_penalty_absolute)
l2_custom = run_experiment(x_train, y_train, x_val, y_val, x_test, y_test,
                           overprediction_penalty_objective, overprediction_penalty_squared)
pr_custom = run_experiment(x_train, y_train, x_val, y_val, x_test, y_test,
                           overprediction_penalty_objective, overprediction_penalty_custom)

temp = pd.DataFrame()
all_winnings = []
all_std_winnings = []
overpreds = []
results = [l1, l2, l1_custom, l2_custom, pr_custom]
for res, name in zip(results, res_names) :
    res.calculate_winnings()
    all_winnings.append(res.winnings)
    all_std_winnings.append(res.winning_percent)
    overpreds.append(res.over_percent)
temp['Cost'] = res_names
temp['Winnings'] = all_winnings
temp['Standardized_Winnings'] = all_std_winnings
temp['% OverPredicted'] = overpreds
temp['Penalty'] = penalty
total_results = pd.concat([total_results, temp], 0)

In [158]:
best_winnings = total_results.sort_values(by='Winnings', ascending=False).drop_duplicates(subset=['Cost'])
best_std_winnings = total_results.sort_values(by='Standardized_Winnings', ascending=False).drop_duplicates(subset=['Cost'])
best_std_winnings

,Cost,Winnings,Standardized_Winnings,% OverPredicted,Penalty
2,Asymmetric MAE Custom,9620.347383,670.328990,0.214249,5
3,Asymmetric MSE Custom,12109.453288,657.519941,0.357774,12
4,Asymmetric Price is Right Eval,13040.864275,591.461058,0.456058,12
0,MAE,13106.620179,518.973499,0.557982,12
1,MSE,12083.902792,332.002559,0.731669,10


In [159]:
best_winnings

,Cost,Winnings,Standardized_Winnings,% OverPredicted,Penalty
4,Asymmetric Price is Right Eval,13298.838315,557.408396,0.486739,10
3,Asymmetric MSE Custom,13213.585046,515.005578,0.549142,3
0,MAE,13106.620179,518.973499,0.557982,3
1,MSE,12083.902792,332.002559,0.731669,5
2,Asymmetric MAE Custom,11387.717411,669.309173,0.315133,3
